In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import torch

def func_Psi(u):
    return torch.log(torch.cosh(torch.norm(u)))

def func_F(matrix_x, list_y):
    out = 0
    for y_k in list_y:
        out += func_Psi(y_k - torch.matmul(matrix_x, y_k))
    return out

def grad_F(matrix_x, list_y):
    matrix_x_clone = matrix_x.clone()
    matrix_x_clone = matrix_x_clone.detach().requires_grad_()
    out = func_F(matrix_x_clone, list_y)
    out.backward()
    return matrix_x_clone.grad

def line_search(alpha_bar, rho, c1, matrix_x, list_y):
    r_t = 0
    while True:
        alpha = alpha_bar * (rho ** r_t)
        grad =  grad_F(matrix_x, list_y)
        lhs = func_F(matrix_x - alpha * grad, list_y)
        rhs = func_F(matrix_x, list_y) - c1 * alpha * (torch.norm(grad) ** 2)
        if lhs <= rhs:
            break
        else:
            r_t += 1
    return alpha

def find_max_abs_of_X(matrix_x):
    out = torch.abs(matrix_x[0][0])
    for rows in matrix_x:
        for entry in rows:
            if torch.abs(entry) > out:
                out = torch.abs(entry)
    return out

def problem_3_5(list_y):
    matrix_x = torch.zeros(list_y.shape[1], list_y.shape[1], dtype = torch.float64)
    t, c1, alpha_bar, rho = 0, 0.01, 0.1, 0.9
    while True:
        alpha = line_search(alpha_bar, rho, c1, matrix_x, list_y)
        matrix_x_new = matrix_x - alpha * grad_F(matrix_x, list_y)
        if func_F(matrix_x, list_y) - func_F(matrix_x_new, list_y) <= 1e-6:
            break
        else:
            matrix_x = matrix_x_new
            t += 1
    print("The value of F at convergence: " + str(func_F(matrix_x_new, list_y).item()))
    print("The largest element of x in absolute value: "+ str(find_max_abs_of_X(matrix_x_new).item()))
    print("The number of required iterations: " + str(t))

In [2]:
if __name__ == "__main__":
    list_y = pd.read_csv('homework2_data/project2_Y.csv')
    list_y = list_y.drop(['Unnamed: 0'], axis=1).to_numpy()
    list_y = torch.tensor(list_y, dtype = torch.float64)
    
    problem_3_5(list_y)

The value of F at convergence: 0.0001609264200027296
The largest element of x in absolute value: 0.7562430492024295
The number of required iterations: 344


In [8]:
def get_q_t(list_l, t, m):
    j = t % m
    q_t = list_l[j+1]
    return q_t

def get_xi_l(l, dims):
    x_i_l = torch.zeros(dims, dims, dtype = torch.float64)
    x_i_l[l[0], l[1]] = 1
    x_i_l[l[1], l[0]] = 1
    return x_i_l

def partial_F(q_t, matrix_x, list_y):
    out = 0
    i, j = q_t[0], q_t[1]
    for y_k in list_y:
        z_k = y_k - torch.matmul(matrix_x, y_k)
        out += (torch.tanh(torch.norm(z_k))/torch.norm(z_k)) * (z_k[i] * y_k[j] + z_k[j] * y_k[i])
    return -out

def problem_3_6(list_y, list_l):
    matrix_x = torch.zeros(list_y.shape[1], list_y.shape[1], dtype = torch.float64)
    t, c1, alpha_bar, rho = 0, 0.01, 1, 0.9
    list_x = []
    m = len(list_l) - 1
    while True:
        q_t = get_q_t(list_l, t, m)
        alpha = line_search(alpha_bar, rho, c1, matrix_x, list_y)
        partial = partial_F(q_t, matrix_x, list_y)
        if torch.abs(partial) >= 1e-8:
            matrix_x = matrix_x - alpha * partial * get_xi_l(q_t, list_y.shape[1])
        list_x.append(matrix_x)
        if t % 100 == 0:
          print("The value of F at step " + str(t) + ": " + str(func_F(matrix_x, list_y).item()))
        if t >= m and func_F(list_x[t - m], list_y) - func_F(list_x[t], list_y) <= 1e-6:
            break
        else:
            t += 1
    print("The value of F at convergence: " + str(func_F(matrix_x, list_y).item()))
    print("The largest element of x in absolute value: "+ str(find_max_abs_of_X(matrix_x).item()))
    print("The number of required iterations: " + str(t))

In [9]:
if __name__ == "__main__":
    list_l = pd.read_csv('homework2_data/project2_C.csv')
    list_l = list_l.drop(['Unnamed: 0'], axis=1).to_numpy()
    list_l = torch.tensor(list_l)
    
    problem_3_6(list_y, list_l)

The value of F at step 0: 43.620559521709644
The value of F at step 100: 41.301439847013675
The value of F at step 200: 38.0635355100751
The value of F at step 300: 34.42411062419935
The value of F at step 400: 32.29601053384496
The value of F at step 500: 29.418292797610444
The value of F at step 600: 25.813478185126307
The value of F at step 700: 23.7629304253249
The value of F at step 800: 23.221621709965877
The value of F at step 900: 22.413552160657854
The value of F at step 1000: 21.623327655136695
The value of F at step 1100: 21.045642492030506
The value of F at step 1200: 20.290504543072583
The value of F at step 1300: 19.658159772679713
The value of F at step 1400: 19.396162961724045
The value of F at step 1500: 19.122656051888242
The value of F at step 1600: 18.77007558377296
The value of F at step 1700: 18.516365588350503
The value of F at step 1800: 18.260942138036594
The value of F at step 1900: 17.930389521341045
The value of F at step 2000: 17.75305258820033
The value of